In [6]:
model_args["observation_dim"]

256

In [3]:
import numpy as np
import uisrnn
import defaultdict


SAVED_MODEL_NAME = 'demo_model.uisrnn'

model_args = {"crp_alpha":1.0, "enable_cuda":True, "observation_dim":256, "rnn_depth":1, "rnn_dropout":0.2, "rnn_hidden_size": 512, "sigma2": None, "transition_bias": None, "verbosity": 2}
training_args = {"batch_size": 10, "enforce_cluster_id_uniqueness": True, "grad_max_norm": 5.0, "learning_rate": 0.001, "num_permutations": 10, "optimizer": 'adam', "regularization_weight": 1e-05, "sigma_alpha": 1.0, "sigma_beta": 1.0, "train_iteration": 20000} 
inference_args = {"beam_size": 10, "look_ahead": 1, "test_iteration": 2}

# def diarization_experiment(model_args, training_args, inference_args):
def diarization_experiment():
    """Experiment pipeline.

    Load data --> train model --> test model --> output result

    Args:
    model_args: model configurations
    training_args: training configurations
    inference_args: inference configurations
    """

    predicted_cluster_ids = []
    test_record = []

    train_data = np.load('../data/uisrnn/toy_training_data.npz', allow_pickle=True)
    test_data = np.load('../data/uisrnn/toy_testing_data.npz', allow_pickle=True)
    train_sequence = train_data['train_sequence']
    train_cluster_id = train_data['train_cluster_id']
    test_sequences = test_data['test_sequences'].tolist()
    test_cluster_ids = test_data['test_cluster_ids'].tolist()

    model = uisrnn.UISRNN(model_args)

    # Training.
    # If we have saved a mode previously, we can also skip training by
    # calling：
    # model.load(SAVED_MODEL_NAME)
    model.fit(train_sequence, train_cluster_id, training_args)
    model.save(SAVED_MODEL_NAME)

    # Testing.
    # You can also try uisrnn.parallel_predict to speed up with GPU.
    # But that is a beta feature which is not thoroughly tested, so
    # proceed with caution.
    for (test_sequence, test_cluster_id) in zip(test_sequences, test_cluster_ids):
        predicted_cluster_id = model.predict(test_sequence, inference_args)
        predicted_cluster_ids.append(predicted_cluster_id)
        accuracy = uisrnn.compute_sequence_match_accuracy(
            test_cluster_id, predicted_cluster_id)
        test_record.append((accuracy, len(test_cluster_id)))
        print('Ground truth labels:')
        print(test_cluster_id)
        print('Predicted labels:')
        print(predicted_cluster_id)
        print('-' * 80)

    output_string = uisrnn.output_result(model_args, training_args, test_record)

    print('Finished diarization experiment')
    print(output_string)


def main():
    """The main function."""
#   model_args, training_args, inference_args = uisrnn.parse_arguments()
#   diarization_experiment(model_args, training_args, inference_args)
    diarization_experiment()
    


In [7]:

model_args, training_args, inference_args = uisrnn.parse_arguments("")
# diarization_experiment(model_args, training_args, inference_args)

TypeError: parse_arguments() takes 0 positional arguments but 1 was given